<a href="https://colab.research.google.com/github/meesumabidi/NAVTCC/blob/main/sir_asif_rfm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  #import modules
  import pandas as pd # for dataframes
  import matplotlib.pyplot as plt # for plotting graphs
  import seaborn as sns # for plotting graphs
  import datetime as dt
  from sklearn import tree
  import squarify 

In [ ]:
data = pd.read_excel("/content/Online_Retail.xlsx")
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data= data[pd.notnull(data['CustomerID'])]
filtered_data=data[['Country','CustomerID']].drop_duplicates()

#Top ten country's customer
filtered_data.Country.value_counts()[:20]


United Kingdom    1087
Germany             31
France              30
Australia            8
Spain                8
Portugal             7
Belgium              5
Italy                4
Switzerland          3
EIRE                 3
Japan                3
Netherlands          2
Austria              2
Sweden               2
Cyprus               2
Iceland              1
Denmark              1
Norway               1
Finland              1
Lithuania            1
Name: Country, dtype: int64

In [ ]:
uk_data=data[data.Country=='United Kingdom']

uk_data.describe ()

,Quantity,UnitPrice,CustomerID
count,35963.000000,35963.000000,35963.000000
mean,10.789005,3.158938,15662.166671
std,557.707895,5.977907,1622.541472
min,-74215.000000,0.000000,12346.000000
25%,1.000000,1.250000,14415.000000
50%,4.000000,2.100000,15581.000000
75%,12.000000,3.750000,17211.000000
max,74215.000000,295.000000,18283.000000


In [ ]:
uk_data = uk_data[(uk_data['Quantity']>0)]
uk_data.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 35089 entries, 0 to 65082
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   InvoiceNo    35089 non-null  object        
 1   StockCode    35089 non-null  object        
 2   Description  35089 non-null  object        
 3   Quantity     35089 non-null  int64         
 4   InvoiceDate  35089 non-null  datetime64[ns]
 5   UnitPrice    35089 non-null  float64       
 6   CustomerID   35089 non-null  float64       
 7   Country      35089 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 2.4+ MB


In [ ]:
  #Filter Required coloumn for RFM Analysis
  uk_data=uk_data[['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]
  uk_data['TotalPrice'] = uk_data['Quantity'] * uk_data['UnitPrice']
  uk_data['InvoiceDate'].min(),uk_data['InvoiceDate'].max()
  PRESENT = dt.datetime(2011,12,10)
  uk_data['InvoiceDate'] = pd.to_datetime(uk_data['InvoiceDate'])

In [ ]:
    #### RFM Analysis #############
    rfm= uk_data.groupby('CustomerID').agg({'InvoiceDate': lambda date: (PRESENT - date.max()).days,
    'InvoiceNo': lambda num: len(num),
    'TotalPrice': lambda price: price.sum()})
    rfm.columns
    # Change the name of columns
    rfm.columns=['monetary','frequency','recency']
    rfm.head()

,monetary,frequency,recency
CustomerID,,,
12346.0,325,1,77183.60
12747.0,323,20,1009.31
12748.0,325,693,4646.90
12820.0,326,11,170.46
12826.0,324,25,367.72


In [ ]:
  #computing the Quantile of RFM value for segmentation
  rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, ['1','2','3','4'])
  rfm['f_quartile'] = pd.qcut(rfm['frequency'], 4, ['4','3','2','1'])
  rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, ['4','3','2','1'])
  rfm.head()

,monetary,frequency,recency,r_quartile,f_quartile,m_quartile
CustomerID,,,,,,
12346.0,325,1,77183.60,4,4,4
12747.0,323,20,1009.31,4,3,4
12748.0,325,693,4646.90,4,1,4
12820.0,326,11,170.46,1,3,4
12826.0,324,25,367.72,3,2,4


In [ ]:
#### Combine RFM Quantile score
rfm['RFM_Score'] = rfm.r_quartile.astype(str)+ rfm.f_quartile.astype(str) + rfm.m_quartile.astype(str)
rfm.head()
# Filter out Top/Best customers as (111)
a = rfm[rfm['RFM_Score']=='222'].sort_values('monetary', ascending=False).head()
print(a)


            monetary  frequency  recency  ... f_quartile m_quartile RFM_Score
CustomerID                                ...                                
13240.0          365         21   224.07  ...          2          2       222
13969.0          365         25   317.59  ...          2          2       222
14896.0          365         32   257.22  ...          2          2       222
17950.0          365         39   263.17  ...          2          2       222
17220.0          364         32   317.50  ...          2          2       222

[5 rows x 7 columns]


In [ ]:
a = titanic.groupby('class')[['survived']].sum().index.get_level_values(0).tolist()
 
d = titanic.groupby('class')[['survived']].sum().reset_index().survived.values.tolist()
 
squarify.plot(sizes=d,label=a, alpha=.8 )
plt.axis('off')
plt.show()

NameError: ignored